<a href="https://colab.research.google.com/github/prasapi/AIML/blob/master/Copy_of_M0_01_KNN_Fruits_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint
## Not for grades

## Learning Objective

The objective of this experiment is to understand Knn classifier.

## Dataset

The dataset chosen for this  experiment is a handmade fruits dataset. The dataset contains 60 records. Each record represents the following details of fruits : 

*  Weight -   It is the mass of an object. With respect to this dataset, we have calculated the weights in grams 

* Sphericity -   is a measure of how closely the shape of an object approaches that of a mathematically perfect sphere.

* Color -  Every fruit has a different color at different stages. You can encode the color to an integer value. For example

     - Orange as 22
     - Greenish Yellow as 23
     - Green as 24
     - Red as 25
     - Reddish Yellow as 26

*  Label -   We have considered two fruits for simplicity. They are Apple and Orange.




### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "2000656" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "6305008936" #@param {type:"string"}


In [5]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook="M0_002" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/fruits_weight_sphercity.csv")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      print("Your submission is successful.")
      print("Ref Id:", submission_id)
      print("Date of submission: ", r["date"])
      print("Time of submission: ", r["time"])
      print("View your submissions: https://iiith-aiml.talentsprint.com/notebook_submissions")
      print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    from IPython.display import HTML
    HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id))
  
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


#### Importing Required Packages

In [0]:
import numpy as np
import matplotlib.pyplot as plt

#### Loading the data

In [0]:
def labels_converter(s):
  s = s.lower()
  if s == "orange":
    return 0
  if s == "apple":
    return 1
  return -1

In [0]:
def color_converter(s):
  s = s.lower()
  if s == 'green':
    return 22
  if s == 'greenish yellow':
    return 23
  if s == 'orange':
    return 24
  if s == 'red':
    return 25
  if s == 'reddish yellow':
    return 26
  return -1
  

In [0]:
import pandas as pd
fruits_data = pd.read_csv("fruits_weight_sphercity.csv",converters={"labels":labels_converter,"Color":color_converter})

To get better understanding of the data let us visualize first five rows of the data using head () and last five rows of the data using tail()

In [10]:
fruits_data.head()

,Color,Weight,Sphericity,labels
0,25,200,0.40,1
1,24,320,1.00,0
2,24,310,0.95,0
3,25,220,0.45,1
4,25,230,0.50,1


In [11]:
fruits_data.tail()

,Color,Weight,Sphericity,labels
64,25,200,0.43,1
65,25,200,0.49,1
66,25,200,0.42,1
67,25,201,0.40,1
68,25,202,0.40,1


In [0]:
# Storing data and labels in two seperate variables
data = fruits_data[["Color","Sphericity","Weight"]]
labels = fruits_data["labels"]

In [0]:
colors =fruits_data[["Color"]]

In [14]:
print(colors)

    Color
0      25
1      24
2      24
3      25
4      25
..    ...
64     25
65     25
66     25
67     25
68     25

[69 rows x 1 columns]


### Splitting the data into train and test sets 

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
 # Training and testing set ratio is 70 : 30
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [17]:
# Let us see the size of train and  test sets
X_train.shape, X_test.shape

((46, 3), (23, 3))

In [18]:
# Let us see first five rows of the training data

X_train[:5]

,Color,Sphericity,Weight
28,22,0.89,330
7,23,0.98,330
61,22,0.68,344
40,25,0.40,204
50,24,0.80,272


### Training a  Knn Classifier 

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
k = 5
neigh = KNeighborsClassifier(n_neighbors=k)

In [55]:
# Training or fitting the model with the train data
neigh.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [59]:
# Testing the trained model
print(neigh.predict(X_test))
print(X_test)

[0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
    Color  Sphericity  Weight
22     22        0.98     326
0      25        0.40     200
47     25        0.25     225
4      25        0.50     230
53     23        0.92     333
18     22        0.68     266
10     23        0.92     333
33     22        0.55     271
44     24        1.00     346
12     22        0.42     259
31     22        0.69     255
9      25        0.53     243
59     23        0.75     310
5      25        0.40     245
68     25        0.40     202
30     22        0.67     269
57     26        0.70     201
35     22        0.59     250
56     25        0.10     200
46     25        0.30     210
16     26        0.48     267
34     22        0.71     269
42     25        0.42     210


In [57]:
# Calculating the score
neigh.score(X_test,y_test)

0.8260869565217391

### Exercise : Try to observe the change in score by changing the k value to 5

### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = " No Comments" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]

In [0]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [0]:
#@title In class Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Inclass_support = " " #@param ["Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [0]:
#@title Online Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Online_support = "" #@param ["Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [0]:
#@title Run this cell to submit your notebook{ vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")